<a href="https://colab.research.google.com/github/Meghna1904/compilerlab/blob/main/LEX_and_YACC_Compiler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LEX and YACC Compiler in Colab

Drawbacks:
* Regular interrupts (Ctrl+D, Ctrl+C) for shell won't work in Colab while inputting for program.
<br>Workaround: Store your inputs in a txt file and pass it to the program.

In [ ]:
#@title Install *prerqeuisites* (run this cell first to work on LEX/YACC)
!sudo apt install flex bison

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libfl-dev libfl2
Suggested packages:
  bison-doc flex-doc
The following NEW packages will be installed:
  bison flex libfl-dev libfl2
0 upgraded, 4 newly installed, 0 to remove and 38 not upgraded.
Need to get 1,072 kB of archives.
After this operation, 3,667 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 flex amd64 2.6.4-8build2 [307 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 bison amd64 2:3.8.2+dfsg-1build1 [748 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 libfl2 amd64 2.6.4-8build2 [10.7 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/main amd64 libfl-dev amd64 2.6.4-8build2 [6,236 B]
Fetched 1,072 kB in 0s (3,391 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend can

## Lex only

In [ ]:
#@title Writing Lex program
%%writefile program.l

%{
    #include <stdio.h>
    int ctChar=0;
    int ctSpace=0;
    int ctWord=0;
    int ctLine=0;
%}
WORD [^ \t\n,\.:]+
EOL [\n]
BLANK [ ]
%%

{WORD} {ctWord++; ctChar+=yyleng;}
{BLANK} {ctSpace++;}
{EOL} {ctLine++;}
. {ctChar++;}
%%

void main(int argc, char *argv[]){
    if(argc!=2){
        printf("Usage:\n\t./a.out <FILENAME>\n");
        exit(0);
    }

    yyin=fopen(argv[1],"r");
    yylex();

    printf("Word Count: %d\n",ctWord);
    printf("Character Count: %d\n",ctChar);
    printf("Space Count: %d\n",ctSpace);
    printf("Line Count: %d\n",ctLine);
    fclose(yyin);

}

int yywrap(){
    return 1;
}

Writing program.l


if you want to use at txt as an input

In [ ]:
%%writefile program.txt

This is a sample file.

Writing program.txt


In [ ]:
#@title Shell Execution (you can rewrite the commands as per your need, eg. if you want to include a file as an input)
%%shell

lex -l program.l
gcc lex.yy.c
./a.out program.txt

Word Count: 5
Character Count: 18
Space Count: 4
Line Count: 2


## Lex and Yacc combined

In [ ]:
#@title Writing YACC program
%%writefile program.y

%{
    #include<stdio.h>
    #include<stdlib.h>
%}
%token DIGIT LETTER UND NL
%%
stmt: variable NL {printf("Valid Identifier\n");exit(0);}
variable: LETTER alphanumeric;
alphanumeric: LETTER alphanumeric
            | DIGIT alphanumeric
            | UND alphanumeric
            | LETTER
            | DIGIT
            | UND;
%%

int yyerror(){
    printf("Invalid Identifier\n");
    exit(0);
}

void main(){
    printf("Enter the variable name: ");
    yyparse();
}

Overwriting program.y


In [ ]:
#@title Writing Lex program
%%writefile program.l

%{
    #include "y.tab.h"
%}
%%
[a-zA-Z] {return LETTER;}
[0-9] {return DIGIT;}
[_] {return UND;}
\n {return NL;}
. {return yytext[0];}
%%

Overwriting program.l


if you want to use at txt as an input

In [ ]:
%%writefile program.txt

This is a sample file.

In [ ]:
m

In [ ]:
#@title Shell Execution (you can rewrite the commands as per your need, eg. if you want to include a file as an input)
%%shell

yacc -d program.y
lex program.l
cc y.tab.c lex.yy.c -ll
./a.out

y.tab.c: In function ‘yyparse’:
y.tab.c:1024:16: warning: implicit declaration of function ‘yylex’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wimplicit-function-declaration-Wimplicit-function-declaration]8;;]
 1024 |       yychar = yylex ();
      |                ^~~~~
y.tab.c:1165:7: warning: implicit declaration of function ‘yyerror’; did you mean ‘yyerrok’? []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wimplicit-function-declaration-Wimplicit-function-declaration]8;;]
 1165 |       yyerror (YY_("syntax error"));
      |       ^~~~~~~
      |       yyerrok
Enter the variable name: name
Valid Identifier


In [ ]:
%%writefile program1.l
%{
    #include "y.tab.h"
%}
%%
"a"     { return A; }
"b"     { return B; }
\n      { return NL; }
.       { return yytext[0]; }
%%
int yywrap() { return 1; }

Overwriting program1.l


In [ ]:
%%writefile program1.y

%{
#include <stdio.h>
#include <stdlib.h>
int yylex(void);
void yyerror(char *s);
%}
%token A B NL
%%
stmt: S NL { printf("Valid string\n"); exit(0); };
S   : A S | B S | A B B;
%%
void yyerror(char *s) { printf("Invalid string\n"); exit(0); }
int main() { printf("Enter string: "); yyparse(); return 0; }

Writing program1.y


In [ ]:
%%shell
yacc -d program1.y
lex program1.l
cc y.tab.c lex.yy.c -ll
./a.out

Enter string: abb
Valid string


In [ ]:
%%writefile program2.l

%{
  #include <stdio.h>
  #include <stdlib.h>
%}

%%
"//"(.*)  {}
"/*"(.|\n)*?"*/" {}
. {ECHO;}
%%

int yywrap() {return 1;}

int main() {
  yylex();
}

Overwriting program2.l


In [ ]:
%%writefile program2.c
int main(){
  //This is program
  printf("My name is Meh");
  /*helo
  hy*/
}

Overwriting program2.c


In [ ]:
%%writefile count.l
%{
#include <stdio.h>
#include <stdlib.h>
int chars=0,lines=0,words=0,sent=0;
%}

%%
[.!?] {sent++;}
[^ \t\n]+ {words++; chars+=yyleng;}
\n {lines++;}
. {chars++;}
%%

int yywrap(){return 1;}

int main(){
    yylex();
    printf("Count of words : %d", words );
    printf("Count of lines : %d", lines );
    printf("Count of character : %d", chars );
    printf("Count of sentences : %d", sent );
}

Overwriting count.l


In [ ]:
%%writefile count.txt
meghna

Overwriting count.txt


In [ ]:
%%shell

lex count.l
gcc lex.yy.c
./a.out < count.txt

Count of words : 6Count of lines : 3Count of character : 23Count of sentences : 1

LEX PRACTICE




> Add blockquote





```
# This is formatted as code
```

# Lex program


# vowel consonants

In [ ]:
%%writefile vowel.l
%{
#include <stdio.h>
int vowel=0,cons=0;
%}

%%
[aeiouAEIOU] {vowel++;}
[a-zA-Z] {cons++;}
%%

int yywrap(){return 1;}

int main(){
  yylex();
  printf("Vowel Count = : %d\n",vowel);
  printf("Consonants Count = : %d\n",cons);
}

Writing vowel.l


In [ ]:
%%shell

lex vowel.l
gcc lex.yy.c
./a.out < count.txt


Vowel Count = : 2
Consonants Count = : 4


# frequency of characters


In [ ]:
%%writefile frequency.l
%{
    #include <stdio.h>
    int freq[256] ={0};
%}

%%
. {freq[yytext[0]]++;}
\n {}
%%

int yywrap(){return 1;}
int main(){
  yylex();
  for(int i=0;i<256;i++){
    if(freq[i]>0){
      printf("%c : %d\n",i,freq[i]);
    }

  }
  return 0;
}

Overwriting frequency.l


In [ ]:
%%shell
lex frequency.l
gcc lex.yy.c
./a.out < count.txt

a : 1
e : 1
g : 1
h : 1
m : 1
n : 1


#Convert substring abc to ABC

In [ ]:
%%writefile substring.l
%{
    #include <stdio.h>

%}

%%
"abc" {printf("ABC");}
.|\n  {ECHO;}
%%

int yywrap(){return 1;}

int main(){
  printf("Enter the string ");
  yylex();
  return 0;

}

Writing substring.l


In [ ]:
%%shell
lex substring.l
gcc lex.yy.c
./a.out

Enter the string abc is happy.
ABC is happy.


CalledProcessError: Command 'lex substring.l
gcc lex.yy.c
./a.out
' died with <Signals.SIGINT: 2>.

# YACC PROGRAMS



#variable declaration

In [ ]:
%%writefile var.l
%{
#include "y.tab.h"
%}

%%
[ \t\n] ;
"int"|"float"|"char"|"arr" {return TYPE;}
[a-zA-Z_][a-zA-Z_0-9]* {return ID;}

";" {return SEMICOLON;}
%%

int yywrap(){return 1;}

Overwriting var.l


In [ ]:
%%writefile var.y
%{
#include <stdio.h>
#include <stdlib.h>
%}

%token ID TYPE SEMICOLON

%%
stmt:TYPE ID SEMICOLON {printf("Valid");exit(0);};
%%

int yyerror(char *s){
  printf("Invalid");
  exit(0);
}
int main(){
  printf("Enter variable declaration\n");
  yyparse();
}


Overwriting var.y


In [ ]:
%%shell
yacc -d var.y
lex var.l
cc y.tab.c lex.yy.c -ll
./a.out

y.tab.c: In function ‘yyparse’:
y.tab.c:1016:16: warning: implicit declaration of function ‘yylex’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wimplicit-function-declaration-Wimplicit-function-declaration]8;;]
 1016 |       yychar = yylex ();
      |                ^~~~~
y.tab.c:1157:7: warning: implicit declaration of function ‘yyerror’; did you mean ‘yyerrok’? []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wimplicit-function-declaration-Wimplicit-function-declaration]8;;]
 1157 |       yyerror (YY_("syntax error"));
      |       ^~~~~~~
      |       yyerrok
Enter variable declaration
int a;
Valid

# LANGUAGE ACCEPTING

#a^nb^n

In [ ]:
%%writefile p1.l
%{
  #include "y.tab.h"
%}

%%
"a" {return A;}
"b" {return B;}
\n {return NL;}
[ \t] ;
%%

int yywrap() { return 1; }

Writing p1.l


In [ ]:
%%writefile p1.y
%{
#include <stdio.h>
#include <stdlib.h>
%}

%token A B NL

%%
stmt: S NL {printf("Valid string"); exit(0);}
S : A S B| A B
%%

int yyerror(char *s){
  printf("Invalid");
}

int main(){
  printf("Enter string:");
  yyparse();
  }

Writing p1.y


In [ ]:
%%shell
yacc -d p1.y
lex p1.l
cc y.tab.c lex.yy.c -ll
./a.out

lex: can't open p1.l
y.tab.c: In function ‘yyparse’:
y.tab.c:1017:16: warning: implicit declaration of function ‘yylex’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wimplicit-function-declaration-Wimplicit-function-declaration]8;;]
 1017 |       yychar = yylex ();
      |                ^~~~~
y.tab.c:1158:7: warning: implicit declaration of function ‘yyerror’; did you mean ‘yyerrok’? []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wimplicit-function-declaration-Wimplicit-function-declaration]8;;]
 1158 |       yyerror (YY_("syntax error"));
      |       ^~~~~~~
      |       yyerrok
Enter string:
Invalid


# a star b star

In [ ]:
%%writefile p2.l
%{
#include "y.tab.h"
%}

%%
"a" {return A;}
"b" {return B;}
\n {return NL;}
[ \t] ;
%%

int yywrap(){
  return 1;
}

Overwriting p2.l


In [ ]:
%%writefile p2.y
%{
  #include <stdio.h>
  #include <stdlib.h>
%}

%token A B NL
%%
stmt:S NL {printf("Valid");exit(0);}
S: X Y
X: A X | ;
Y: B Y | ;
%%

int yyerror(char *s){
  printf("invalid");
  exit(0);
}
int main(){
  printf("Enter a string: ");
  yyparse();
}

Overwriting p2.y


In [ ]:
%%shell
yacc -d p2.y
lex p2.l
gcc y.tab.c lex.yy.c
./a.out

y.tab.c: In function ‘yyparse’:
y.tab.c:1022:16: warning: implicit declaration of function ‘yylex’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wimplicit-function-declaration-Wimplicit-function-declaration]8;;]
 1022 |       yychar = yylex ();
      |                ^~~~~
y.tab.c:1163:7: warning: implicit declaration of function ‘yyerror’; did you mean ‘yyerrok’? []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wimplicit-function-declaration-Wimplicit-function-declaration]8;;]
 1163 |       yyerror (YY_("syntax error"));
      |       ^~~~~~~
      |       yyerrok
Enter a string: 
Valid

#Calculator


In [ ]:
%%writefile calc.l
%{
 #include "y.tab.h"
%}

%%
[0-9]+ {yylval=atoi(yytext);return DIGIT;}
[ \t] {}
\n {return NL;}
. {return yytext[0];}
%%

int yywrap(){
  return 1;
}

Overwriting calc.l


In [ ]:
%%writefile calc.y
%{
  #include <stdio.h>
  #include <stdlib.h>
%}

%token DIGIT OP NL
%left '+' '-'
%left '*' '/' '%'
%%
stmt: E NL {printf("Result:%d", $1); exit(0);}
E: E '+' T {$$ = $1+$3; }
 | E '-' T {$$ = $1-$3;}
 |T {$$=$1;} ;
T: T '*' V {$$ = $1*$3;}
 | T '/' V {if($3==0){
          printf("Division by zero\n");
          exit(0);
          }
          else{$$ = $1/$3;}}
 |V {$$=$1;};
V:'(' E ')' {$$=$2;}
 |DIGIT {$$=$1;};

%%

int yyerror(char *s){
  printf("Invalid Expression");
  exit(0);
}
int main(){
  printf("Enter Expression : \n");
  yyparse();
}

Overwriting calc.y


In [ ]:
%%shell
yacc -d calc.y
lex calc.l
gcc y.tab.c lex.yy.c
./a.out

y.tab.c: In function ‘yyparse’:
y.tab.c:1035:16: warning: implicit declaration of function ‘yylex’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wimplicit-function-declaration-Wimplicit-function-declaration]8;;]
 1035 |       yychar = yylex ();
      |                ^~~~~
y.tab.c:1228:7: warning: implicit declaration of function ‘yyerror’; did you mean ‘yyerrok’? []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wimplicit-function-declaration-Wimplicit-function-declaration]8;;]
 1228 |       yyerror (YY_("syntax error"));
      |       ^~~~~~~
      |       yyerrok
Enter Expression : 


CalledProcessError: Command 'yacc -d calc.y
lex calc.l 
gcc y.tab.c lex.yy.c
./a.out
' died with <Signals.SIGINT: 2>.

In [ ]:
%%writefile cal1.l
%{
  #include "y.tab.h"
%}

%%
[0-9]+ {yylval=atoi(yytext);return DIGIT;}
[ \t] ;
\n {return NL;}
. {return yytext[0];}
%%
int yywrap(){return 1;}

In [ ]:
%%writefile cal1.y
%{
  #include <stdio.h>
  #include <stdlib.h>
%}

%token DIGIT NL
%left '+' '-'
%left '*' '/' '%'

%%
stmt:E NL {printf("Result: %d",$1);exit(0);}
E:E '+' T {$$=$1+$3;}
 |E '-' T {$$=$1+$3;}
 |T {$$=$1;}
T:T '*' V {$$=$1*$3;}
 |T '/' V {if($3==0){
          printf("Division by zero");
          exit(0);
          }
          else{$$=$1+$3;}
  }
 |T '%' V { $$ = $1 % $3; }
 |V{$$=$1;}
V:'(' E ')' {$$=$2;}
 | DIGIT {$$=$1;}
%%

int yyerror(char *s){
  printf("invalid expression");
  exit(0);
}

int main(){
  printf("Enter Expression:");
  yyparse();
}

In [ ]:
%%shell
yacc -d cal1.y
lex cal1.l
gcc y.tab.c lex.yy.c
./a.out

# For LOOP


In [ ]:
%%writefile for1.l
%{
#include "y.tab.h"
%}
letter [A-Za-z]
digit [0-9]
%%
for { return FOR; }
int { return INT; }
[a-zA-Z_][a-zA-Z_0-9]* { return ID; }
(("+"|"-")?)({digit}+) { return NUM; }
"=" { return EQUALS; }
"<"|"<="|">"|">="|"=="|"!=" { return
REL_OP; }
"++"|"--" { return INC_DEC; }
"(" { return PAR_OPEN; }
")" { return PAR_CLOSE; }
"," { return COMMA; }
";" { return SEMICOLON; }
\n { return NL; }
. {}
%%
int yywrap() {
return 1;
}


Overwriting for1.l


In [ ]:
%%writefile for1.y
%{
#include <stdio.h>
#include <stdlib.h>
%}
%token FOR INT ID NUM EQUALS REL_OP
INC_DEC PAR_OPEN PAR_CLOSE COMMA
SEMICOLON NL
%%
stmt: S NL { printf("Valid\n"); exit(0); };
S: FOR PAR_OPEN INIT COND ACTION
PAR_CLOSE;
INIT: INT ID EQUALS NUM SEMICOLON;
COND: ID REL_OP NUM SEMICOLON
| ID REL_OP ID SEMICOLON
;
ACTION: ID INC_DEC;
%%
int yyerror(char *msg) {
printf("Invalid\n");
exit(0);
}
int main() {
printf("Enter the string: ");
yyparse();
}


Overwriting for1.y


In [ ]:
%%shell
yacc -d for1.y
lex for1.l
gcc y.tab.c lex.yy.c
./a.out

y.tab.c: In function ‘yyparse’:
y.tab.c:1058:16: warning: implicit declaration of function ‘yylex’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wimplicit-function-declaration-Wimplicit-function-declaration]8;;]
 1058 |       yychar = yylex ();
      |                ^~~~~
y.tab.c:1199:7: warning: implicit declaration of function ‘yyerror’; did you mean ‘yyerrok’? []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wimplicit-function-declaration-Wimplicit-function-declaration]8;;]
 1199 |       yyerror (YY_("syntax error"));
      |       ^~~~~~~
      |       yyerrok
Enter the string: for(int i=0; i<10; i++)
Valid


# SWITCH CASE


In [ ]:
%%writefile switch1.l
%{
#include "y.tab.h"
%}

letter [A-Za-z]
digit [0-9]
%%
"switch" {return SWITCH;}
"case" {return CASE;}
"default" {return DEFAULT;} /* Added rule for default keyword */
({letter}|(_))({letter}|{digit}|(_))* { return ID; }
[0-9]+  { return NUM; }
"="     { return EQUALS; }
";"     { return SEMICOLON; }
":"     { return COLON; }
"("     { return PAR_OPEN; }
")"     { return PAR_CLOSE; }
"{"     { return CURLY_OPEN; }
"}"     { return CURLY_CLOSE; }
break   { return BREAK; }
\n      { return NL; }
[ \t]   ;
.       {}
%%

int yywrap() { return 1; }

Overwriting switch1.l


In [ ]:
%%writefile switch1.y
%{
#include <stdio.h>
#include <stdlib.h>
%}

%token SWITCH CASE DEFAULT COLON ID NUM BREAK SEMICOLON PAR_OPEN PAR_CLOSE CURLY_OPEN CURLY_CLOSE EQUALS NL

%%

stmt: SWITCH PAR_OPEN ID PAR_CLOSE CURLY_OPEN case_list optional_default CURLY_CLOSE { printf("Valid\n"); exit(0); }
    | SWITCH PAR_OPEN ID PAR_CLOSE CURLY_OPEN case_list optional_default CURLY_CLOSE NL { printf("Valid\n"); exit(0); }
    ;


case_list: /* empty */
         | case_list CASE NUM COLON ACTION BREAK SEMICOLON
         ;

optional_default: /* empty */
                | DEFAULT COLON ACTION BREAK SEMICOLON
                ;

ACTION: ID EQUALS NUM
      | ID EQUALS ID
      ;

%%

int yyerror(char *msg) {
    printf("Invalid\n");
    exit(0);
}

int main() {
    printf("Enter the string: ");
    yyparse();
}

Overwriting switch1.y


In [ ]:
%%shell
yacc -d switch1.y
lex switch1.l
gcc y.tab.c lex.yy.c
./a.out

switch1.l:20: warning, rule cannot be matched
y.tab.c: In function ‘yyparse’:
y.tab.c:1065:16: warning: implicit declaration of function ‘yylex’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wimplicit-function-declaration-Wimplicit-function-declaration]8;;]
 1065 |       yychar = yylex ();
      |                ^~~~~
y.tab.c:1206:7: warning: implicit declaration of function ‘yyerror’; did you mean ‘yyerrok’? []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wimplicit-function-declaration-Wimplicit-function-declaration]8;;]
 1206 |       yyerror (YY_("syntax error"));
      |       ^~~~~~~
      |       yyerrok
Enter the string: switch(x){ case 1: y=5; break; case 2: y=10; break; default: y=0; break; }
Invalid


# Backend


In [ ]:
%%writefile backend.c

#include <stdio.h>
#include <string.h>
int main() {

    FILE *f = fopen("input.txt", "r");
    if (f == NULL) return 1;

    char res[3], op[2], op1[2], op2[2], eq[2];
    while (fscanf(f, "%s %s %s %s %s", res, eq, op1, op, op2) != EOF) {
        printf("MOV AX, [%s]\n", op1);
        switch (op[0]) {
            case '+': printf("ADD AX, [%s]\n", op2); break;
            case '-': printf("SUB AX, [%s]\n", op2); break;
            case '*': printf("MOV BX, [%s]\nMUL BX\n", op2); break;
            case '/': printf("MOV BX, [%s]\nDIV BX\n", op2); break;
        }
        printf("MOV [%s], AX\n\n", res);
    }
    fclose(f);
    return 0;
}

Overwriting backend.c


In [ ]:
%%writefile input.txt
X = a / b
Y = X * c

Overwriting input.txt


In [ ]:
%%shell
gcc backend.c
./a.out

MOV AX, [a]
MOV BX, [b]
DIV BX
MOV [X], AX

MOV AX, [X]
MOV BX, [c]
MUL BX
MOV [Y], AX



# Constant Propagation


In [ ]:
%%writefile cons_prop.c
#include <stdio.h>
#include <string.h>
#include <stdlib.h>
#include <ctype.h>
struct expr {
    char op[2], op1[5], op2[5], res[5];
    int flag;
} arr[10];
int n;
void input() {
    printf("\nEnter the max expressions: ");
    scanf("%d", &n);
    printf("Enter the input (op op1 op2 res):\n");
    for (int i = 0; i < n; i++) {
        scanf("%s %s %s %s", arr[i].op, arr[i].op1, arr[i].op2, arr[i].res);
        arr[i].flag = 0;
    }
}
void output() {
    printf("\nOptimized code:\n");
    for (int i = 0; i < n; i++) {
        if (!arr[i].flag) {
            printf("%s %s %s %s\n", arr[i].op, arr[i].op1, arr[i].op2, arr[i].res);
        }
    }
}
void change(int p, char *res) {
    for (int i = p + 1; i < n; i++) {
        if (strcmp(arr[p].res, arr[i].op1) == 0) strcpy(arr[i].op1, res);
        if (strcmp(arr[p].res, arr[i].op2) == 0) strcpy(arr[i].op2, res);
    }
}
void constant() {
    int op1, op2, res; char res1[5];
    for (int i = 0; i < n; i++) {
        if (isdigit(arr[i].op1[0]) && isdigit(arr[i].op2[0]) || strcmp(arr[i].op, "=") == 0) {
            op1 = atoi(arr[i].op1); op2 = atoi(arr[i].op2);
            switch (arr[i].op[0]) {
                case '+': res = op1 + op2; break;
                case '-': res = op1 - op2; break;
                case '*': res = op1 * op2; break;
                case '/': res = op1 / op2; break;
                case '=': res = op1; break;
            }
            sprintf(res1, "%d", res);
            arr[i].flag = 1;
            change(i, res1);
        }
    }
}
int main() { input(); constant(); output(); return 0; }

Overwriting cons_prop.c


In [ ]:
%%shell
gcc cons_prop.c
./a.out


Enter the max expressions: 4
Enter the input (op op1 op2 res):
= 3 - a
+ a b t1
+ a c t2
+ t1 t2 t3

Optimized code:
+ 3 b t1
+ 3 c t2
+ t1 t2 t3


# First and follow



In [ ]:
%%writefile first.c
#include <stdio.h>
#include <string.h>
#include <ctype.h>

#define MAX 20

char prod[MAX][MAX];       // all productions
int n;                     // number of productions
char nonTerminals[MAX];    // list of non-terminals
int ntCount = 0;           // count of non-terminals

// check if symbol is a non-terminal
int isNonTerminal(char c) {
    return isupper(c);
}

// add a symbol to a set if it's not already present
void addToSet(char set[], char c) {
    if (!strchr(set, c)) {
        int len = strlen(set);
        set[len] = c;
        set[len + 1] = '\0';
    }
}

// ---------------- FIND FIRST ----------------
void findFirst(char result[], char c) {
    // If c is a terminal, FIRST(c) = {c}
    if (!isNonTerminal(c)) {
        addToSet(result, c);
        return;
    }

    // Otherwise, go through all productions with LHS = c
    for (int i = 0; i < n; i++) {
        if (prod[i][0] == c) {
            // Loop through RHS (starts from index 2)
            for (int j = 2; prod[i][j] != '\0'; j++) {
                char sym = prod[i][j];

                // If epsilon directly
                if (sym == '#') {
                    addToSet(result, '#');
                    break;
                }

                char temp[50] = "";
                findFirst(temp, sym);

                // Add all symbols from FIRST(sym) except epsilon
                for (int k = 0; temp[k] != '\0'; k++) {
                    if (temp[k] != '#')
                        addToSet(result, temp[k]);
                }

                // If epsilon not found, stop checking next symbols
                if (!strchr(temp, '#'))
                    break;

                // If we reached end and epsilon found everywhere
                if (prod[i][j + 1] == '\0')
                    addToSet(result, '#');
            }
        }
    }
}

// ---------------- FIND FOLLOW ----------------
void findFollow(char result[], char c) {
    // Rule 1: start symbol gets $
    if (prod[0][0] == c)
        addToSet(result, '$');

    // Check every production
    for (int i = 0; i < n; i++) {
        // Traverse RHS
        for (int j = 2; prod[i][j] != '\0'; j++) {
            if (prod[i][j] == c) {

                // Case 1: symbol followed by something
                if (prod[i][j + 1] != '\0') {
                    char next = prod[i][j + 1];
                    char temp[50] = "";
                    findFirst(temp, next);

                    // Add all FIRST(next) except epsilon
                    for (int k = 0; temp[k] != '\0'; k++) {
                        if (temp[k] != '#')
                            addToSet(result, temp[k]);
                    }

                    // If epsilon in FIRST(next), FOLLOW(LHS) also added
                    if (strchr(temp, '#'))
                        findFollow(result, prod[i][0]);
                }

                // Case 2: symbol is at end → FOLLOW(LHS)
                else if (prod[i][0] != c) {
                    findFollow(result, prod[i][0]);
                }
            }
        }
    }
}

// ---------------- MAIN FUNCTION ----------------
int main() {
    printf("Enter number of productions: ");
    scanf("%d", &n);

    printf("Enter productions (e.g., E=TR, R=+TR, R=#):\n");
    for (int i = 0; i < n; i++) {
        scanf("%s", prod[i]);
        // store non-terminals
        if (!strchr(nonTerminals, prod[i][0])) {
            nonTerminals[ntCount++] = prod[i][0];
        }
    }

    printf("\n--- FIRST and FOLLOW Sets ---\n");
    for (int i = 0; i < ntCount; i++) {
        char c = nonTerminals[i];
        char firstRes[50] = "";
        char followRes[50] = "";

        findFirst(firstRes, c);
        findFollow(followRes, c);

        printf("FIRST(%c) = { ", c);
        for (int j = 0; firstRes[j]; j++)
            printf("%c ", firstRes[j]);
        printf("}\n");

        printf("FOLLOW(%c) = { ", c);
        for (int j = 0; followRes[j]; j++)
            printf("%c ", followRes[j]);
        printf("}\n\n");
    }

    return 0;
}



Overwriting first.c


In [ ]:
%%shell
gcc first.c
./a.out

Enter number of productions: 8
Enter productions (e.g., E=TR, R=+TR, R=#):
E=TR
R=+TR
R=#
T=FY
Y=*FY
Y=#
F=(E)
F=i

--- FIRST and FOLLOW Sets ---
FIRST(E) = { ( i }
FOLLOW(E) = { $ ) }

FIRST(R) = { + # }
FOLLOW(R) = { $ ) }

FIRST(T) = { ( i }
FOLLOW(T) = { + $ ) }

FIRST(Y) = { * # }
FOLLOW(Y) = { + $ ) }

FIRST(F) = { ( i }
FOLLOW(F) = { * + $ ) }



# E Closure

In [ ]:
%%writefile e_epsilon.c
#include <stdio.h>

int arr[10][10], visited[10];
int n, t, k, i, j, s1, s2;
char trans;

void closure(int i) {
    visited[i] = 1;
    for (j = 0; j < n; j++) {
        if (arr[i][j] == 1 && visited[j] != 1) {
            printf(",q%d", j);
            closure(j);
        }
    }
}

int main() {
    printf("\nEnter the no of states: ");
    scanf("%d", &n);

    printf("Enter number of transitions:");
    scanf("%d", &t);

    printf("Enter transitions (e.g., 0 e 1):\n");
    for (i = 0; i < t; i++) {
        scanf("%d %c %d", &s1, &trans, &s2);
        if (trans == 'e') {
            arr[s1][s2] = 1;
        }
    }

    printf("\nThe epsilon closures are: ");
    for (i = 0; i < n; i++) {
        printf("\nState q%d: {q%d", i, i);
        for (k = 0; k < n; k++) {
            visited[k] = 0;
        }
        closure(i);
        printf("}");
    }

    printf("\n");
    return 0;
}

Writing e_epsilon.c


In [ ]:
%%shell
gcc e_epsilon.c
./a.out


Enter the no of states: 4
Enter number of transitions:3
Enter transitions (e.g., 0 e 1):
0 e 1
1 e 2
2 1 3

The epsilon closures are: 
State q0: {q0,q1,q2}
State q1: {q1,q2}
State q2: {q2}
State q3: {q3}


# Epsilon NFA to NFA

In [34]:
%%writefile enfa_nfa.c
#include <stdio.h>
#include <string.h>

char enfa[20][3], final[30];
int ntrans;

int isin(char c, char str[]) {
    for (int i = 0; i < strlen(str); i++)
        if (str[i] == c)
            return 1;
    return 0;
}

void add(char str[], char c) {
    if (!isin(c, str)) {
        int len = strlen(str);
        str[len] = c;
        str[len + 1] = '\0';
    }
}

void addstate(char c1, char c2) {
    for (int i = 0; i < ntrans; i++)
        if (enfa[i][0] == c2 && enfa[i][1] != 'e')
            printf("%c %c %c\n", c1, enfa[i][1], enfa[i][2]);
        else if (enfa[i][0] == c2 && enfa[i][1] == 'e' && enfa[i][2] != c1)
            addstate(c1, enfa[i][2]);
}

int main() {
    int i;
    printf("Enter number of transitions:");
    scanf("%d", &ntrans);

    while (getchar() != '\n');

    printf("Enter transitions as \nstate symbol state\n");
    for (i = 0; i < ntrans; i++) {
        scanf(" %c %c %c", &enfa[i][0], &enfa[i][1], &enfa[i][2]);
        while (getchar() != '\n');
    }

    printf("Final states:");
    scanf("%s", final);

    printf("\nNFA transitions without epsilon\n");
    for (i = 0; i < ntrans; i++)
        if (enfa[i][1] != 'e')
            printf("%c %c %c\n", enfa[i][0], enfa[i][1], enfa[i][2]);
        else
            addstate(enfa[i][0], enfa[i][2]);

    for (i = 0; i < ntrans; i++)
        if (isin(enfa[i][2], final) && enfa[i][1] == 'e')
            add(final, enfa[i][0]);

    printf("Final states: {%s}\n", final);
    return 0;
}

Writing enfa_nfa.c


In [35]:
%%shell
gcc enfa_nfa.c
./a.out

Enter number of transitions:1
Enter transitions as 
state symbol state
0 e 1
Final states:1

NFA transitions without epsilon
Final states: {10}


# shift reducer


In [20]:
%%writefile shift.c
#include <stdio.h>
#include <string.h>

char s[256], b[256];
int t = 0, p = 0;

void printTable(const char *action) {
    printf("%-20s %-20s$ %-20s\n", s, b + p, action);
}

void R() {
    int changed = 1;
    while (changed) {
        changed = 0;

        // (E) -> E
        if (t >= 3 && s[t-3] == '(' && s[t-2] == 'E' && s[t-1] == ')') {
            t -= 3;
            s[t++] = 'E';

            printTable("REDUCE (E)->E");
            changed = 1;
            continue;
        }

        // E+E -> E
        if (t >= 3 && s[t-3] == 'E' && s[t-2] == '+' && s[t-1] == 'E') {
            t -= 3;
            s[t++] = 'E';

            printTable("REDUCE E+E->E");
            changed = 1;
            continue;
        }

        // E*E -> E
        if (t >= 3 && s[t-3] == 'E' && s[t-2] == '*' && s[t-1] == 'E') {
            t -= 3;
            s[t++] = 'E';

            printTable("REDUCE E*E->E");
            changed = 1;
            continue;
        }

        // i -> E
        if (t >= 1 && s[t-1] == 'i') {
            s[t-1] = 'E';

            printTable("REDUCE i->E");
            changed = 1;
            continue;
        }
    }
}

int main() {
    printf("GRAMMAR:\n");
    printf("E -> E+E | E*E | (E) | i\n\n");

    printf("Enter input: ");
    if (scanf("%255s", b) != 1) return 0;

    printf("\n%-20s %-20s %-20s\n", "STACK", "INPUT", "ACTION");
    printf("------------------------------------------------------------\n");

    while (b[p] != '\0') {
        s[t++] = b[p++];

        printTable("SHIFT");
        R();
    }

    R();

    if (t == 1 && s[0] == 'E')
        printf("\nResult: Input accepted.\n");
    else {
        printf("\nResult: Input rejected.\n");
        printf("Final Stack: %s\n", s);
    }

    return 0;
}


Overwriting shift.c


In [24]:
%%shell
gcc shift.c
./a.out

GRAMMAR:
E -> E+E | E*E | (E) | i

Enter input: i+i+(i)*i

STACK                INPUT                ACTION              
------------------------------------------------------------
i                    +i+(i)*i            $ SHIFT               
E                    +i+(i)*i            $ REDUCE i->E         
E+                   i+(i)*i             $ SHIFT               
E+i                  +(i)*i              $ SHIFT               
E+E                  +(i)*i              $ REDUCE i->E         
E+E                  +(i)*i              $ REDUCE E+E->E       
E+E                  (i)*i               $ SHIFT               
E+(                  i)*i                $ SHIFT               
E+(i                 )*i                 $ SHIFT               
E+(E                 )*i                 $ REDUCE i->E         
E+(E)                *i                  $ SHIFT               
E+EE)                *i                  $ REDUCE (E)->E       
E+EE)                *i                  $ REDUCE

# Intermediate code generation

In [33]:
%%writefile inter.c

#include <stdio.h>
#include <string.h>
#include <ctype.h>
#include <stdlib.h>

#define MAX 100
char stack[MAX];
int top = -1;

void push(char c) {
   if (top < MAX - 1)
   top++;
   stack[top] = c;
    }
char pop() {
  if (top > -1)
  return stack[top--];
  }
int priority(char c) {
    if(c == '^') return 3;
    if(c == '*' || c == '/') return 2;
    if(c == '+' || c == '-') return 1;
    return 0;
}
void infixToPostfix(char infix[], char postfix[]) {
    int i, j = 0;
    for (i = 0; infix[i]; i++) {
        if (isalnum(infix[i]))
        postfix[j++] = infix[i];

        else if (infix[i] == '(')
        push(infix[i]);

        else if (infix[i] == ')') {
            while (top != -1 && stack[top] != '(')
            postfix[j++] = pop();
            pop(); // Pop '('
        } else {
            while (top != -1 && priority(stack[top]) >= priority(infix[i]))
                postfix[j++] = pop();
            push(infix[i]);
        }
    }
    while (top != -1) postfix[j++] = pop();
    postfix[j] = '\0';
}
void threeadd(char *str) {
    int t = 1;
    for (int i = 0; str[i]; i++) {
        if (isalnum(str[i])) {
            push(str[i]);
        } else {
            char op2 = pop();
            char op1 = pop();
            printf("t%d := %c %c %c\n", t, op1, str[i], op2);
            push('0' + t);
            t++;
        }
    }
}
int main() {
    char infix[MAX], postfix[MAX];
    printf("Enter a simple expression: ");
    scanf("%s", infix);
    infixToPostfix(infix, postfix);
    threeadd(postfix);
    return 0;
}

Overwriting inter.c


In [32]:
%%shell
gcc inter.c
./a.out

Enter a simple expression: a+b*c-d
t1 := b * c
t2 := a + 1
t3 := 2 - d
